In [1]:
import pandas as pd

In [3]:
df = pd.read_excel("warehouse_inventory_data.xlsx")

In [5]:
df.head()

,SKU_ID,Warehouse_ID,Category,Stock_Level,Reorder_Point,Avg_Daily_Sales,Shelf_Life_Days,Last_Order_Date,Order_Fulfillment_Days,Stockout_Occurred,Return_Rate,Labor_Cost_Per_Day
0,SKU-0041,WH-1,Fresh Produce,51,38,2.24,180,2023-05-21,5,0,0.12,189.17
1,SKU-0015,WH-2,Bakery,86,20,4.86,7,2023-12-13,5,0,0.12,241.61
2,SKU-0038,WH-4,Fresh Produce,21,30,9.71,7,2023-02-17,4,0,0.19,100.16
3,SKU-0015,WH-5,Frozen Foods,59,30,6.37,7,2023-10-15,2,0,0.10,186.39
4,SKU-0046,WH-5,Beverages,48,36,6.31,14,2023-08-18,2,1,0.01,294.75


In [7]:
# Check data type first
print(df.dtypes)

SKU_ID                            object
Warehouse_ID                      object
Category                          object
Stock_Level                        int64
Reorder_Point                      int64
Avg_Daily_Sales                  float64
Shelf_Life_Days                    int64
Last_Order_Date           datetime64[ns]
Order_Fulfillment_Days             int64
Stockout_Occurred                  int64
Return_Rate                      float64
Labor_Cost_Per_Day               float64
dtype: object


In [9]:
# Check for null values
print(df.isnull().sum())

SKU_ID                    0
Warehouse_ID              0
Category                  0
Stock_Level               0
Reorder_Point             0
Avg_Daily_Sales           0
Shelf_Life_Days           0
Last_Order_Date           0
Order_Fulfillment_Days    0
Stockout_Occurred         0
Return_Rate               0
Labor_Cost_Per_Day        0
dtype: int64


In [11]:
# Check for duplicate rows
print(f"Duplicate rows: {df.duplicated().sum()}")

Duplicate rows: 0


In [15]:
# Derived fields

# Days to stockout
df['days_to_stockout'] = df['Stock_Level'] / df['Avg_Daily_Sales']

In [17]:
# Derived field: is slow moving

df['is_slow_moving'] = df['Avg_Daily_Sales'] < 5

In [19]:
df.head()

,SKU_ID,Warehouse_ID,Category,Stock_Level,Reorder_Point,Avg_Daily_Sales,Shelf_Life_Days,Last_Order_Date,Order_Fulfillment_Days,Stockout_Occurred,Return_Rate,Labor_Cost_Per_Day,days_to_stockout,is_slow_moving
0,SKU-0041,WH-1,Fresh Produce,51,38,2.24,180,2023-05-21,5,0,0.12,189.17,22.767857,True
1,SKU-0015,WH-2,Bakery,86,20,4.86,7,2023-12-13,5,0,0.12,241.61,17.695473,True
2,SKU-0038,WH-4,Fresh Produce,21,30,9.71,7,2023-02-17,4,0,0.19,100.16,2.162719,False
3,SKU-0015,WH-5,Frozen Foods,59,30,6.37,7,2023-10-15,2,0,0.10,186.39,9.262166,False
4,SKU-0046,WH-5,Beverages,48,36,6.31,14,2023-08-18,2,1,0.01,294.75,7.606973,False


In [21]:
# EDA Metrics

# % slow-moving SKUs
df['is_slow_moving'].value_counts(normalize=True)

is_slow_moving
False    0.522
True     0.478
Name: proportion, dtype: float64

In [25]:
# Stockout count

df[df['Stock_Level'] == 0].shape[0]

5

In [27]:
# Overstock count

df[df['Stock_Level'] > 2 * df['Reorder_Point']].shape[0]

240

In [29]:
# Avg fulfillment time

df['Order_Fulfillment_Days'].mean()

3.548

In [39]:
# Labour cost trend (groupby month and sum labor cost)

df['month'] = df['Last_Order_Date'].dt.to_month('M')
labor_cost_by_month = df.groupby('month')['Labor_Cost_Per_Day'].sum().reset_index()

AttributeError: 'DatetimeProperties' object has no attribute 'to_month'

In [33]:
# Group by warehouse

warehouse_summary = df.groupby('Warehouse_ID')[['Stock_Level', 'Avg_Daily_Sales', 'Order_Fulfillment_Days', 'Labor_Cost_Per_Day']].mean()

In [35]:
df.head()

,SKU_ID,Warehouse_ID,Category,Stock_Level,Reorder_Point,Avg_Daily_Sales,Shelf_Life_Days,Last_Order_Date,Order_Fulfillment_Days,Stockout_Occurred,Return_Rate,Labor_Cost_Per_Day,days_to_stockout,is_slow_moving,month
0,SKU-0041,WH-1,Fresh Produce,51,38,2.24,180,2023-05-21,5,0,0.12,189.17,22.767857,True,2023-05
1,SKU-0015,WH-2,Bakery,86,20,4.86,7,2023-12-13,5,0,0.12,241.61,17.695473,True,2023-12
2,SKU-0038,WH-4,Fresh Produce,21,30,9.71,7,2023-02-17,4,0,0.19,100.16,2.162719,False,2023-02
3,SKU-0015,WH-5,Frozen Foods,59,30,6.37,7,2023-10-15,2,0,0.10,186.39,9.262166,False,2023-10
4,SKU-0046,WH-5,Beverages,48,36,6.31,14,2023-08-18,2,1,0.01,294.75,7.606973,False,2023-08


In [37]:
df.to_csv("cleaned_warehouse_inventory_data.csv", index=False)